

# Dooooooooooom - project - quick start

Create a virtual environment
```bash
python -m venv venv
```

Activate the virtual environment

On macOS/Linux:
```bash
source venv/bin/activate
```

On Windows (Command Prompt):
```bash
venv\Scripts\activate
```
```bash
pip install vizdoom
```

Run below example cell 


### Example cell spinning up a game instance and feeding the game look random actions


In [9]:
import vizdoom as vzd
import os
import random
import numpy as np

# Create a game instance
game = vzd.DoomGame()

# Find the scenarios directory in the VizDoom package
scenarios_path = os.path.join(os.path.dirname(vzd.__file__), 'scenarios')
print(f"VizDoom scenarios directory: {scenarios_path}")

# Check if scenarios directory exists and list available scenarios
if os.path.exists(scenarios_path):
    scenarios = [f for f in os.listdir(scenarios_path) if f.endswith('.cfg')]
    print(f"Available scenario configs: {scenarios}")
else:
    print("Scenarios directory not found!")

# Load basic scenario configuration file
basic_config_path = os.path.join(scenarios_path, 'basic.cfg')

if os.path.exists(basic_config_path):
    game.load_config(basic_config_path)
    print(f"Loaded config from: {basic_config_path}")
else:
    print(f"Config file not found at: {basic_config_path}")
    print("Setting up manual configuration...")
    
    # Manual configuration if config file not found
    # Set the scenario WAD file path
    basic_wad_path = os.path.join(scenarios_path, 'basic.wad')
    game.set_doom_scenario_path(basic_wad_path)
    game.set_doom_map("map01")
    
    # Configure screen and rendering
    game.set_screen_resolution(vzd.ScreenResolution.RES_640X480)
    game.set_screen_format(vzd.ScreenFormat.RGB24)
    game.set_render_hud(False)
    game.set_render_crosshair(True)
    game.set_render_weapon(True)
    game.set_render_decals(False)
    game.set_render_particles(False)
    
    # Add available buttons for the basic scenario
    game.add_available_button(vzd.Button.MOVE_LEFT)
    game.add_available_button(vzd.Button.MOVE_RIGHT)
    game.add_available_button(vzd.Button.MOVE_FORWARD)
    game.add_available_button(vzd.Button.MOVE_BACKWARD)
    game.add_available_button(vzd.Button.ATTACK)
    game.add_available_button(vzd.Button.ALTATTACK)
    game.add_available_button(vzd.Button.USE)
   
    
    # Set other game parameters
    game.set_episode_timeout(200)
    game.set_window_visible(True)
    game.set_living_reward(-1)

# Initialize the game
try:
    game.init()
    print("VizDoom initialized successfully!")
    print(f"Available actions: {len(game.get_available_buttons())} buttons")
    
    for i in range(1000):
        # Get the state
        state = game.get_state()
        
        if state:
            # Get screen buffer
            screen = state.screen_buffer
            print(f"Frame {i}: Screen shape: {screen.shape}")
            
            # Perform a random action
            # Get number of available buttons
            n_buttons = len(game.get_available_buttons())
            
            # Completely random actions
            action = [random.choice([True, False]) for _ in range(n_buttons)]
            
            
            reward = game.make_action(action)
            print(f"Frame {i}: Action {action}, Reward: {reward}")
            
            if game.is_episode_finished():
                print("Episode finished, starting new episode...")
                game.new_episode()
        else:
            print("No state available, breaking...")
            break
    
    game.close()
    print("Game closed successfully.")
    
except Exception as e:
    print(f"Error initializing VizDoom: {e}")
    game.close()

VizDoom scenarios directory: /Users/dimitridumont/code/skool/ML_510/.venv/lib/python3.11/site-packages/vizdoom/scenarios
Available scenario configs: ['my_way_home.cfg', 'learning.cfg', 'cig.cfg', 'simpler_basic.cfg', 'multi.cfg', 'predict_position.cfg', 'multi_duel.cfg', 'health_gathering.cfg', 'oblige.cfg', 'health_gathering_supreme.cfg', 'rocket_basic.cfg', 'defend_the_center.cfg', 'take_cover.cfg', 'deadly_corridor.cfg', 'defend_the_line.cfg', 'deathmatch.cfg', 'basic.cfg']
Loaded config from: /Users/dimitridumont/code/skool/ML_510/.venv/lib/python3.11/site-packages/vizdoom/scenarios/basic.cfg
VizDoom initialized successfully!
Available actions: 3 buttons
Frame 0: Screen shape: (3, 240, 320)
Frame 0: Action [False, False, True], Reward: -1.0
Frame 1: Screen shape: (3, 240, 320)
Frame 1: Action [True, True, True], Reward: -1.0
Frame 2: Screen shape: (3, 240, 320)
Frame 2: Action [False, False, True], Reward: -1.0
Frame 3: Screen shape: (3, 240, 320)
Frame 3: Action [False, True, Fals